In [ ]:
import sys
sys.path.append("src")
from qaoa.core.QAOA import QAOArunner
from  qaoa.models.MaxCutProblem import MaxCutProblem
import  qaoa.models.params as params
from rustworkx.visualization import mpl_draw as draw_graph
from old_jupyter_vizualisers.load_data import load_graph_from_csv
import numpy as np


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors
import networkx as nx
import rustworkx as rx   # only for the converter
problem = MaxCutProblem()
graphs = problem.get_erdos_renyi_graphs_paper1()
# --- utility unchanged --------------------------------------------------------
def rwx_to_nx(rwx_g: rx.PyGraph) -> nx.Graph:
    H = nx.Graph()
    for n in rwx_g.node_indices():
        H.add_node(n, weight=rwx_g[n])
    H.add_edges_from(rwx_g.edge_list())
    return H


# --- colour-map limits --------------------------------------------------------
all_w = [gr[n] for gr in graphs for n in gr.node_indices()]
norm, cmap = colors.Normalize(min(all_w), max(all_w)), plt.cm.plasma

# ----------------------------------------------------------------------------- 
# ❶  create a GRID of 2×2 axes PLUS ONE extra axis for the colour-bar
# ----------------------------------------------------------------------------- 
fig = plt.figure(figsize=(12, 6.8), dpi=300)

gs = fig.add_gridspec(2, 3, width_ratios=[1, 1, 0.04], wspace=0.05, hspace=0.25)
axes = [fig.add_subplot(gs[i//2, i%2]) for i in range(4)]       # graphs
cax   = fig.add_subplot(gs[:, 2])                               # colour-bar axis

# ----------------------------------------------------------------------------- 
# ❷  plot the four graphs (identical to previous reply)
# ----------------------------------------------------------------------------- 
for idx, g_rwx in enumerate(graphs):
    G  = rwx_to_nx(g_rwx)
    ax = axes[idx]

    w   = [d["weight"] for _, d in G.nodes(data=True)]
    pos = nx.spring_layout(G, seed=42)

    nx.draw_networkx_edges(G, pos, ax=ax, edge_color="0.70", width=0.8)
    nx.draw_networkx_nodes(G, pos, ax=ax,
                           node_color=cmap(norm(w)),
                           node_size=500, linewidths=0.8, edgecolors="black")
    nx.draw_networkx_labels(
        G, pos,
        labels={n: f"{wt:.2f}" for n, wt in zip(G.nodes, w)},
        font_size=8, font_color="white",
        ax=ax
    )
    ax.set_title(f"Graph {idx+1}", fontsize=12)
    ax.set_axis_off()

# ----------------------------------------------------------------------------- 
# ❸  vertical shared colour-bar, occupying the dedicated axis
# ----------------------------------------------------------------------------- 
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, cax=cax, orientation="vertical")
cbar.set_label("Node weight", fontsize=11)
cbar.ax.tick_params(labelsize=9)

plt.tight_layout(rect=[0, 0, 1, 1])
plt.show()

# optional high-resolution / vector save
fig.savefig("maxcut_graphs_weight_gradient_vertbar.svg", bbox_inches="tight")
